In [56]:
# Made by NaturalStupldity

In [57]:
import os
import random
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import fashion_mnist
from keras.utils import to_categorical

In [58]:
## 1. Session preparation

In [59]:
def set_random_state(seed: int):
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    tf.debugging.set_log_device_placement(False)
    config = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

def clear_session():
    tf.keras.backend.clear_session()

def preparation():
    set_random_state(seed=42)
    clear_session()

In [60]:
## 2. Data preparation

In [61]:
def load_data():
    return fashion_mnist.load_data()

def one_hot_encoding(y_train, y_val, number_of_classes: int = 10):
    return to_categorical(y_train, number_of_classes), to_categorical(y_val, number_of_classes)

def normalize_images(x_train, x_val, pixel_depth: int = 255):
    return x_train.astype('float32') / pixel_depth, x_val.astype('float32') / pixel_depth

def flatten_images(x_train, x_val):
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
    x_val = x_val.reshape(x_val.shape[0], x_val.shape[1] * x_val.shape[2])

    return x_train, x_val

def preprocess_data(x_train, y_train, x_val, y_val):
    x_train, x_val = normalize_images(x_train=x_train, x_val=x_val)
    y_train, y_val = one_hot_encoding(y_train=y_train, y_val=y_val)

    return x_train, y_train, x_val, y_val

def build_dataset():
    (x_train, y_train), (x_val, y_val) = load_data()

    return preprocess_data(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val)

In [62]:
## 3. Building the model

In [63]:
def build_perceptron(input_dimension: int = 784,
                     output_dimension: int = 10,
                     activation_function: str = "relu",
                     loss_function="categorical_crossentropy",
                     optimizer="adam",
                     metrics=None):
    if metrics is None:
        metrics = ["accuracy"]

    model = Sequential()
    model.add(Dense(units=input_dimension, activation=activation_function))
    model.add(Dense(units=78, activation=activation_function))
    model.add(Dense(units=output_dimension, activation="softmax"))
    model.compile(loss=loss_function, optimizer=optimizer, metrics=metrics)

    return model

In [64]:
## 4. Training the model

In [82]:
def train_model(model, x_train, y_train, x_val, y_val, batch_size: int, epochs: int):
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

def evaluate_model(model, x_val, y_val):
    return model.evaluate(x_val, y_val)

def train(x_train, y_train, x_val, y_val, batch_size: int = 64, epochs: int = 10, verbose: bool = True):
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5,
                                         weight_decay=1e-4)

    model = build_perceptron(optimizer=optimizer)
    train_model(model=model, x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val, batch_size=batch_size,epochs=epochs)
    results = evaluate_model(model=model, x_val=x_val, y_val=y_val)
    if verbose:
        print("Validation loss: {}, Validation accuracy: {}".format(results[0], results[1]))

    return results

In [83]:
## 5. Main function

In [86]:
def main():
    preparation()
    x_train, y_train, x_val, y_val = build_dataset()
    x_train, x_val = flatten_images(x_train=x_train, x_val=x_val)
    train(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val, batch_size=16, epochs=10)

In [87]:
main()

2023-10-16 21:11:25.796467: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-16 21:11:25.796496: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/10
   7/3750 [..............................] - ETA: 33s - loss: 2.3342 - accuracy: 0.1339  

2023-10-16 21:11:26.694263: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3746/3750 [============================>.] - ETA: 0s - loss: 0.8879 - accuracy: 0.7143

2023-10-16 21:11:52.891990: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3750/3750 [==============================] - 29s 8ms/step - loss: 0.8875 - accuracy: 0.7145 - val_loss: 0.6392 - val_accuracy: 0.7854
Epoch 2/10
3750/3750 [==============================] - 28s 7ms/step - loss: 0.5746 - accuracy: 0.8074 - val_loss: 0.5599 - val_accuracy: 0.8083
Epoch 3/10
3750/3750 [==============================] - 29s 8ms/step - loss: 0.5183 - accuracy: 0.8244 - val_loss: 0.5313 - val_accuracy: 0.8130
Epoch 4/10
3750/3750 [==============================] - 28s 8ms/step - loss: 0.4891 - accuracy: 0.8350 - val_loss: 0.5049 - val_accuracy: 0.8236
Epoch 5/10
3750/3750 [==============================] - 28s 8ms/step - loss: 0.4718 - accuracy: 0.8401 - val_loss: 0.4938 - val_accuracy: 0.8286
Epoch 6/10
3750/3750 [==============================] - 29s 8ms/step - loss: 0.4590 - accuracy: 0.8436 - val_loss: 0.4875 - val_accuracy: 0.8297
Epoch 7/10
3750/3750 [==============================] - 29s 8ms/step - loss: 0.4496 - accuracy: 0.8471 - val_loss: 0.4806 - val_accuracy: 0.8

2023-10-16 21:16:12.603674: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 2s 6ms/step - loss: 0.4622 - accuracy: 0.8388
Validation loss: 0.46216729283332825, Validation accuracy: 0.8388000130653381
